In [3]:
import pandas as pd
import numpy as np
import torch
import os
import torch.nn.functional as F
from torch.utils.data import DataLoader

# 引入你的配置和工具
import config
from models import LSTMFeatureExtractor, LabelClassifier
from utility_uad_svm import load_data, make_sequences_for_svm, lstm_standardization_train_pre_svm, SeqDataset

def main_lstm_inference():
    # 1. 设备设置
    device = torch.device(config.DEVICE)
    print(f"--- [LSTM Inference] 正在使用设备: {device} ---")

    # ==========================================
    # 2. 加载模型 (G_f 和 G_y)
    # ==========================================
    print("\n--- [步骤 1] 加载训练好的模型 ---")
    
    # 实例化特征提取器 G_f
    G_f = LSTMFeatureExtractor(
        input_size=config.LSTM_INPUT_SIZE,
        hidden_size=config.LSTM_HIDDEN_SIZE,
        num_layers=config.LSTM_NUM_LAYERS,
        dropout=config.LSTM_DROPOUT
    ).to(device)
    
    # 实例化分类器 G_y
    G_y = LabelClassifier(
        input_size=config.LSTM_HIDDEN_SIZE,
        num_classes=config.NUM_CLASSES,
        hidden_dim=config.CLASSIFIER_HIDDEN_DIM
    ).to(device)

    # 加载权重
    gf_path = os.path.join(config.MODEL_SAVE_DIR, "G_f_final.pth")
    gy_path = os.path.join(config.MODEL_SAVE_DIR, "G_y_final.pth")
    
    if not os.path.exists(gf_path) or not os.path.exists(gy_path):
        print("❌ 错误：找不到模型权重文件，请先运行 DANN 训练！")
        return

    G_f.load_state_dict(torch.load(gf_path, map_location=device))
    G_y.load_state_dict(torch.load(gy_path, map_location=device))
    
    # 【关键】设置为评估模式 (关闭 Dropout/BatchNorm 更新)
    G_f.eval()
    G_y.eval()
    print("✅ 模型加载成功。")

    # ==========================================
    # 3. 数据准备 (加载 -> 标准化 -> 序列化)
    # ==========================================
    print("\n--- [步骤 2] 准备目标域数据 ---")
    
    # 3.1 加载原始数据
    # 这里我们只需要目标域数据，所以 src_ids 随便传，只要 tgt_id 对就行
    _, df_tgt = load_data(config.SRC_IDS, config.TGT_ID, config.CSV_PATHS)
    
    if df_tgt is None: return
    
    # 3.2 备份原始数据 (用于最后保存结果)
    df_tgt_raw = df_tgt.copy()
    
    # 清洗
    if df_tgt.isnull().values.any(): df_tgt = df_tgt.fillna(0)
    df_tgt = df_tgt.replace([np.inf, -np.inf], 0)

    # 3.3 标准化 (必须加载 DANN 训练时的 Scaler)
    print(" -> 应用 DANN 训练时的标准化参数...")
    df_tgt = lstm_standardization_train_pre_svm(
        df_tgt, 
        config.FEATURES, 
        config.MODEL_SAVE_DIR
    )

    # 3.4 制作序列
    # 我们复用 make_sequences_for_svm，因为它能返回索引 I，方便我们把预测结果填回原表
    print(" -> 制作时间序列...")
    X_tgt, _, _, TGT_INDICES = make_sequences_for_svm(
        df=df_tgt,
        features=config.FEATURES,
        target=None, # 无标签预测
        seq_len=config.SEQ_LEN,
        step=config.STEP
    )
    
    if len(X_tgt) == 0:
        print("❌ 错误：没有生成有效序列。")
        return

    print(f" -> 序列制作完成。样本数: {len(X_tgt)}")

    # ==========================================
    # 4. 执行预测
    # ==========================================
    print("\n--- [步骤 3] 执行 LSTM 推理 ---")
    
    # 创建 DataLoader (为了显存安全，批量预测)
    # y 传 None 即可
    ds_tgt = SeqDataset(X_tgt, y=None)
    dl_tgt = DataLoader(ds_tgt, batch_size=config.BATCH_SIZE, shuffle=False)
    
    all_preds = []
    all_probs_good = [] # 存储 "是好信号" 的概率
    
    with torch.no_grad(): # 这一步不需要计算梯度，省内存
        for X_batch, _ in dl_tgt:
            X_batch = X_batch.to(device)
            
            # 1. 提取特征
            features = G_f(X_batch)
            
            # 2. 分类 (输出 Logits)
            logits = G_y(features)
            
            # 3. 计算概率 (Softmax)
            # dim=1 表示在类别维度计算。输出形状 [batch, 2]
            # [:, 0] 是好信号概率，[:, 1] 是坏信号概率 (假设 0=Good, 1=Bad)
            probs = F.softmax(logits, dim=1)
            
            # 4. 获取硬分类结果 (最大概率对应的索引)
            preds_batch = torch.argmax(logits, dim=1)
            
            # 收集结果
            all_preds.append(preds_batch.cpu().numpy())
            # 收集 "好信号(Class 0)" 的概率，用于后续 Soft-WLS 权重
            all_probs_good.append(probs[:, 0].cpu().numpy())

    # 合并批次结果
    final_preds = np.concatenate(all_preds)
    final_probs_good = np.concatenate(all_probs_good)

    # ==========================================
    # 5. 结果统计与保存
    # ==========================================
    print("\n--- [步骤 4] 统计与保存 ---")
    
    # 统计比例
    ratio_bad = np.sum(final_preds == 1) / len(final_preds)
    print(f" -> 预测为坏信号 (1) 的比例: {ratio_bad:.2%}")
    print(f" -> 预测为好信号 (0) 的比例: {1-ratio_bad:.2%}")
    
    # 物理合理性检查
    if 0.10 <= ratio_bad <= 0.40:
        print("✅ 结果合理：符合 Aachen 环境约 80% 好信号的先验知识。")
    else:
        print("⚠️ 结果警告：比例可能偏离预期，请检查标准化或模型收敛情况。")

    # 保存回原始 DataFrame
    # 初始化全为 NaN
    df_tgt_raw['predicted_multipath'] = np.nan
    df_tgt_raw['prob_score'] = np.nan # 新增一列：LSTM认为它是好信号的概率
    
    # 根据索引填入
    # 注意：序列化会丢掉前 seq_len-1 个点，所以必须用 TGT_INDICES 对齐
    df_tgt_raw.loc[TGT_INDICES, 'predicted_multipath'] = final_preds
    df_tgt_raw.loc[TGT_INDICES, 'prob_score'] = final_probs_good
    
    # 剔除开头没法预测的 NaN 行 (可选，看你解算器是否健壮)
    # df_result = df_tgt_raw.dropna(subset=['predicted_multipath']) 
    
    # 路径设置
    save_dir = os.path.join(config.MODEL_SAVE_DIR, "results_lstm")
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, "tgt_data_with_lstm_predictions.csv")
    
    df_tgt_raw.to_csv(save_path, index=False)
    print(f"✅ 预测结果已保存至: {save_path}")
    print(f"   (包含列 'predicted_multipath' 和 'prob_score')")

if __name__ == "__main__":
    main_lstm_inference()

--- [LSTM Inference] 正在使用设备: cpu ---

--- [步骤 1] 加载训练好的模型 ---
✅ 模型加载成功。

--- [步骤 2] 准备目标域数据 ---
[load_data] 正在加载目标域数据: 0
[load_data] 目标域加载完成。形状: (12614, 14)
[load_data] 注：源域数据将在 'create_all_sequences' 中逐个加载以保证时序独立性。
 -> 应用 DANN 训练时的标准化参数...
 -> 制作时间序列...
  - X 形状: (12538, 5, 5), y 形状: (0,), S 形状: (12538,)
 -> 序列制作完成。样本数: 12538

--- [步骤 3] 执行 LSTM 推理 ---

--- [步骤 4] 统计与保存 ---
 -> 预测为坏信号 (1) 的比例: 17.54%
 -> 预测为好信号 (0) 的比例: 82.46%
✅ 结果合理：符合 Aachen 环境约 80% 好信号的先验知识。
✅ 预测结果已保存至: c:\Users\yangj\Desktop\GNSS-main\GNSS-main\Transfer Learning\SVM\checkpoints\results_lstm\tgt_data_with_lstm_predictions.csv
   (包含列 'predicted_multipath' 和 'prob_score')
